In [2]:
#1

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(100)

# Sample sentence to learn
sample_sentence = "The Potters, that's right, that's what I heard yes, their son, Harry"
char_set = list(set(sample_sentence))
dic = {c: i for i, c in enumerate(char_set)}

dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size
unit_sequence_length = dic_size

print(dic_size)

input_batch = []
target_batch = []

for i in range(0, len(sample_sentence) - unit_sequence_length):
    x_str = sample_sentence[i:i + unit_sequence_length]
    y_str = sample_sentence[i + 1:i + unit_sequence_length + 1]
    input_batch.append([dic[c] for c in x_str])
    target_batch.append([dic[c] for c in y_str])

x_one_hot = [np.eye(dic_size)[x] for x in input_batch]
X = torch.FloatTensor(np.array(x_one_hot))
Y = torch.LongTensor(np.array(target_batch))
print(X.shape)
print(Y.shape)

class Custom_RNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(Custom_RNN, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers)
        self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

learning_rate = 0.05
training_epochs = 100
model = Custom_RNN(input_size, hidden_size, dic_size, 2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(training_epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs.reshape(-1, dic_size), Y.reshape(-1))
    loss.backward()
    optimizer.step()

    if epoch % 10 == 9:
        print('epoch: ', epoch, 'loss: ', loss.item())

        results = outputs.data.numpy().argmax(axis=2)
        pred__ = ""

        for j, result in enumerate(results):
            if j == 0:
                pred__ += ''.join([char_set[t] for t in result])
            else:
                pred__ += char_set[result[-1]]

        check = 0
        acc = 0

        for a in range(len(sample_sentence) - 1):
            if sample_sentence[a] == pred__[a]:
                check += 1
        acc = check / len(pred__)

print(sample_sentence)
print(pred__)
print("ACC=", acc)

20
torch.Size([48, 20, 20])
torch.Size([48, 20])
epoch:  9 loss:  1.4039795398712158
epoch:  19 loss:  0.27943679690361023
epoch:  29 loss:  0.10503122955560684
epoch:  39 loss:  0.04420391097664833
epoch:  49 loss:  0.0277821384370327
epoch:  59 loss:  0.020924275740981102
epoch:  69 loss:  0.018211375921964645
epoch:  79 loss:  0.016994526609778404
epoch:  89 loss:  0.01633032225072384
epoch:  99 loss:  0.015903938561677933
The Potters, that's right, that's what I heard yes, their son, Harry
herroteers  reetes right, that's what I heard yes, their son, Harry
ACC= 0.014925373134328358


In [3]:
#2

! pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
import random
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

dataset = load_dataset("sepidmnorozy/Korean_sentiment")
test_data = dataset["test"]

# 랜덤 샘플 100개 선택
def get_random_samples(seed, data, n=100):
    random.seed(seed)
    indices = random.sample(range(len(data)), n)
    return [data[i] for i in indices]

# 모델
models = [
    "WhitePeak/bert-base-cased-Korean-sentiment",
    "beomi/kcbert-base",
    "snunlp/KR-Medium",
    ]

# 평가 함수
def evaluate_model(model_name, sample_data):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    sentiment_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

    correct = 0
    for item in sample_data:
        prediction = sentiment_pipeline(item["text"])[0]["label"]
        true_label = "LABEL_1" if item["label"] == 1 else "LABEL_0"
        if prediction == true_label:
            correct += 1
    return correct / len(sample_data)

# 총 3회 평가
num_trials = 3
results = {model_name: [] for model_name in models}

for trial in range(num_trials):
    print(f"\n--- Trial {trial + 1} ---")

    sample_data = get_random_samples(seed=42 + trial, data=test_data)

    for model_name in models:
        print(f"Evaluating {model_name}...")
        accuracy = evaluate_model(model_name, sample_data)
        results[model_name].append(accuracy)
        print(f"Accuracy for {model_name}: {accuracy:.2%}")

# 결과 출력
print("\n--- Final Results ---")
average_results = {}
for model_name, accuracies in results.items():
    avg_accuracy = sum(accuracies) / num_trials
    average_results[model_name] = avg_accuracy
    print(f"{model_name}: {avg_accuracy:.2%} (average over {num_trials} trials)")

# 우수 모델 출력
best_model = max(average_results, key=average_results.get)
print(f"\nBest model: {best_model} with accuracy {average_results[best_model]:.2%}")


--- Trial 1 ---
Evaluating WhitePeak/bert-base-cased-Korean-sentiment...
Accuracy for WhitePeak/bert-base-cased-Korean-sentiment: 81.00%
Evaluating beomi/kcbert-base...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy for beomi/kcbert-base: 49.00%
Evaluating snunlp/KR-Medium...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-Medium and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy for snunlp/KR-Medium: 39.00%

--- Trial 2 ---
Evaluating WhitePeak/bert-base-cased-Korean-sentiment...
Accuracy for WhitePeak/bert-base-cased-Korean-sentiment: 77.00%
Evaluating beomi/kcbert-base...


...

Accuracy for snunlp/KR-Medium: 49.00%

--- Trial 3 ---
Evaluating WhitePeak/bert-base-cased-Korean-sentiment...
Accuracy for WhitePeak/bert-base-cased-Korean-sentiment: 73.00%
Evaluating beomi/kcbert-base...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy for beomi/kcbert-base: 46.00%
Evaluating snunlp/KR-Medium...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-Medium and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy for snunlp/KR-Medium: 43.00%

--- Final Results ---
WhitePeak/bert-base-cased-Korean-sentiment: 77.00% (average over 3 trials)
beomi/kcbert-base: 46.00% (average over 3 trials)
snunlp/KR-Medium: 43.67% (average over 3 trials)

Best model: WhitePeak/bert-base-cased-Korean-sentiment with accuracy 77.00%
